# Pregunta 2


## Funciones de utilidad
### randint, randrange
Utilizamos funciones aleatorias de la libreria random para generar los primos P y Q.

### _power
Retorna "a" elevado a "b" en modulo n. Se asume que a y b son positivos. Utiliza exponenciacion rapida.

### _mcd
Retorna el maximo común divisor entre a y b. Utiliza el metodo de euclides.

In [966]:
from random import randint, randrange

In [967]:
def _power(a, b, n):
    res = 1
    a = a % n
    while b > 0:
        if b % 2 == 1:
            res = (res * a) % n
        b = b // 2
        a = (a * a) % n
    return res


def _mcd(a: int, b: int) -> list[int, int, int]:
    if a == 0:
        return (b, 0, 1)

    g, y, x = _mcd(b % a, a)
    return (g, x - (b // a) * y, y)


def _inversa_modular(a: int, N: int) -> int:
    g, x, _ = _mcd(a, N)
    if g != 1:
        raise Exception("No hay inversa modular")
    return x % N

## RSAReceiver
Esta clase describe a la persona que crea un par de llaves, publica(e, N) y privada(d, N), y es capaz de desencriptar los mensajes que otras personas encriptaron con su llave publica. El protocolo que Utiliza es RSA. 

### Generacion de Primos
El protocolo RSA necesita de dos primos P y Q. Para generar estos, utilizamos numeros aleatorios dentro del rango bit_len entregado. Utilizamos el test de rabin_miller para verificar si un numero es primo o no. Este es un test aleatorizado por lo que no determina 100% la primalidad, pero es capaz de siempre detectar si un numero es compuesto. Como es aleatorizado, lo repetimos varias veces para reducir la probabilidad de error. Aseguramos tambien que P y Q sean distintos.

### Claves
Una vez generados los primos, tenemos que crear nuestras llaves publicas y privadas. Para esto calculamos los números N = P * Q y Phi = (P-1)*(Q-1) utilizando estos primos. Luego, queremos encontrar dos numeros, e y d de manera que (e * d) % Phi == 1, es decir, e y d son inversos en modulo phi. En esta parte tenemos que utilizar la funcion de maximo común divisor (utiliza el metodo de euclides) para asegurar que e sea coprimos con phi y luego encontramos la inversa modular de e en phi. Con estos números, creamos nuestra llave privada (d, N) que guardamos en variables de clase y creamos una llave publica (e, N) que se puede entregar a travez del metodo get_public_key. La llave pública es expresada en un "bytearray" de python en formato PEM.

### Desencriptación
Para desencriptar utilizamos un proceso simple. Al recibir un texto cifrado, lo dividimos en bloques de largo de la cantidad de bytes que se necesitan para representar N. Agregamos 1 a este largo ya que sabemos que la función de encriptación necesita un byte extra en cada bloque para evitar overflow. Para cada bloque, lo transformamos en un numero entero y lo exponenciamos con nuestra clave privada (d, N). Luego, transformamos este numero de vuelta a un array de bytes y despues a un string. Al terminar este proceso tenemos un string m que corresponde al texto cifrado desencriptado.

In [968]:
class RSAReceiver:

    def __init__(self, bit_len: int) -> None:
        self._bit_len = bit_len
        self._P = self._generate_prime()
        Q = self._generate_prime()
        while True:
            if Q != self._P:
                self._Q = Q
                break
            Q = self._generate_prime()
        self._N = self._P * self._Q
        self._public_key = self._make_public_key()
        self._block_length = ((len(bin(self._N)) - 2) // 8) + 1

    
    def _rabin_miller(self, p: int, d: int) -> bool:
        # Elegir testigo
        a = 2 + randint(1, p - 4)
        x = _power(a, d, p)

        if x == 1 or x == (p - 1):
            return True
        
        while d != p - 1:
            x = (x * x) % p
            d *= 2
            if x == 1:
                return False
            if x == p - 1:
                return True

        return False

    
    def _is_prime(self, p: int, k: int) -> bool:
        # Casos Borde
        if p <= 1 or p == 4: # negativo o 1 o 4
            return False
        if p <= 3: # 2 o 3
            return True
        
        # Encontrar d para rabin-miller
        # p = d * 2^r + 1, d impar
        d = p - 1
        while d % 2 == 0:
            d //= 2
        
        for _ in range(k):
            if self._rabin_miller(p, d) == False:
                return False
        return True

    
    def _generate_prime(self) -> int:
        counter = 0
        while counter <= 2 ** (self._bit_len):
            candidate = randrange(2 ** (self._bit_len - 1), (2 ** self._bit_len) - 1)
            counter += 1
            if self._is_prime(candidate, 10):
                return candidate


    def _make_public_key(self) -> bytearray:
        phi = (self._P - 1) * (self._Q - 1)
        e = 2
        while e < phi:
            is_coprime, _, _ = _mcd(e, phi)
            if  is_coprime == 1:
                break
            e += 1
        self._e, self._d = e, _inversa_modular(e, phi)

        len_e = 1 + (len(bin(self._e)) - 2) // 8
        len_e_bytes = bytearray(len_e.to_bytes(4, "big"))
        e_bytes = bytearray(self._e.to_bytes(len_e, "big"))

        len_n = 1 + (len(bin(self._N)) - 2) // 8
        len_n_bytes = bytearray(len_n.to_bytes(4, "big"))
        n_bytes = bytearray(self._N.to_bytes(len_n, "big"))

        public_key = len_e_bytes + e_bytes + len_n_bytes + n_bytes
        return public_key
    

    def get_public_key(self) -> bytearray:
        return self._public_key

    
    def decrypt(self, ciphertext: bytearray) -> str:
        s = ""
        block_amount = len(ciphertext) // self._block_length
        if len(ciphertext) < self._N:
            block_amount = 1

        for i in range(block_amount):
            try:
                s += self._decrypt_block(ciphertext[i * self._block_length: (i + 1) * self._block_length])
            except UnicodeError:
                print("Mensaje mal encriptado")
                return ""

        return s
    

    def _decrypt_block(self, block: bytearray) -> str:
        c = int.from_bytes(block, "big")
        m = pow(c, self._d, self._N)
        barray = bytearray(m.to_bytes(self._block_length + 1, "big"))
        return barray.decode()


## RSASender

Esta clase describe a una persona que quiere enviar mensajes encriptados al RSAReceiver que solo el pueda desencriptar.

### Llave Pública

Esta clase recibe como parametro la llave pública de tipo bytearray en formato PEM. de este bytearray se extraen los valores e y N y se guardan como enteros en variables de clase.

### Encriptación

Para encriptar un mensaje este primero se pasa de un string a un bytearray. Luego, este bytearray es dividido en bloques de largo de la cantidad de bytes de N. Para cada uno de los bloques del mensaje, se transforma a un entero, se realiza la exponenciación de este entero con e y luego se transforma de vuelta a un bytearray. Debido a que estamos exponenciando, cada bloque podria representar un número de hasta un byte más grande. Por esto, cada bloque encriptado tiene un byte de largo más que los bloques del mensaje original. 

In [969]:
class RSASender:
    
    def __init__(self, public_key: bytearray) -> None:
        self.public_key = public_key
        self._make_e_and_N()
        self._block_length = (len(bin(self._n)) - 2) // 8


    def _make_e_and_N(self) -> None:
        e_len = int.from_bytes(self.public_key[:4], "big")
        self._e = int.from_bytes(self.public_key[4:4+e_len], "big")
        n_len = int.from_bytes(self.public_key[4 + e_len: 8 + e_len], "big")
        self._n = int.from_bytes(self.public_key[8 + e_len:8 + e_len + n_len], "big")

    
    def encrypt(self, message: str) -> bytearray:
        mess_bytes = bytearray(message, 'utf-8')
        mess_len = len(mess_bytes)
        block_amount = mess_len // self._block_length
        has_last_block = mess_len % self._block_length != 0

        # First block
        c = self._encrypt_block(mess_bytes[:self._block_length])

        # Middle blocks
        for i in range(1, block_amount):
            c += self._encrypt_block(mess_bytes[i * self._block_length: (i + 1) * self._block_length])

        # Last block
        if has_last_block:
            c += self._encrypt_block(mess_bytes[self._block_length * block_amount:])
        
        return c
    

    def _encrypt_block(self, block: bytearray) -> bytearray:
        m = int.from_bytes(block, "big")
        c = _power(m, self._e, self._n)
        return bytearray(c.to_bytes(self._block_length + 1, "big"))

In [970]:
if __name__ == "__main__":
    message1 = "this is a message"
    print(message1)
    rsaR = RSAReceiver(128)
    rsaS = RSASender(rsaR.get_public_key())
    c = rsaS.encrypt(message1)
    print(c)
    m = rsaR.decrypt(c)
    print(m)

this is a message
bytearray(b'\t\xaf\xf2tm\xd1\x16+`\xca\x94\x8b\xe0\xdd\xda\x1c\xca\x80\x93*\x16\xd9\xa8\xc9rPjR\xcb\x93\x0f8\t\xaf\xf2tm\xd1\x16+`\xca\x94\x8b\xe0\xdd\xda\x1c\xca\x80\x93*\x16\xd9\xa8\xc9rPjR\xcb\x93\x0f8')
                this is a message
